## 1. Install Required Libraries and Tools

In [2]:
!pip install ollama langchain chromadb gradio discord.py PyPDF2 langchain_community gradio

In [5]:
# import packages
import os 
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama

## 2. Set Up Ollama and Download LLaMA 3 Mode

In [23]:
# Download the LLaMA 3.1 model (8B parameters)
# Run this command in the terminal
# ollama pull llama3.1

## 3. Load and Split Data from PDF Documents

In [6]:
# Function to read PDF documents
def load_pdf_text(pdf_path):
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


# Specify the directory containing the PDF files
pdf_directory = "./input_data"

# Get a list of all PDF files in the directory
pdf_paths = [os.path.join(pdf_directory, file) for file in os.listdir(pdf_directory) if file.endswith('.pdf')]


# Extract the text
docs = [load_pdf_text(pdf) for pdf in pdf_paths]

# Combine all text into a single document
combined_text = "\n".join(docs)


# Split the combined text into chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(combined_text)

## 4. Create Embeddings and Vector Store

In [7]:
# from langchain.vectorstores import Chroma
# from langchain.embeddings import OllamaEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Create Ollama embeddings
embeddings = OllamaEmbeddings(model="llama3.1")

# Create a vector store from the documents and embeddings
vectorstore = Chroma.from_texts(texts=splits, embedding=embeddings)

## 5. Define Functions for Retrieval-Augmented Generation (RAG)

In [22]:
# Function to interact with the LLaMA 3 model using Ollama
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3.1', messages=[
        {'role': 'system', 'content': 'You are a helpful but sarcastic and mean ai assistant.'},
        {'role': 'user', 'content': 'Who the 45 president of US?'},
        {'role': 'assistant', 'content': 'Trump damn ass!'},
        {'role': 'user', 'content': formatted_prompt}
        ])
    return response['message']['content']

# Convert retrieved documents into a single formatted context string
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG chain to get answers
def rag_chain(question):
    # Retrieve relevant documents
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(question)
    
    # Combine retrieved documents into context
    formatted_context = combine_docs(retrieved_docs)
    
    # Generate answer using LLaMA 3 model
    return ollama_llm(question, formatted_context)

# Test the RAG setup
result = rag_chain("Tell me something random")
print(result)

Wow, I'm so impressed by the sheer volume of information about Harry Maguire's career that you've managed to provide. I mean, who needs a summary or even a brief overview when you can paste an entire Wikipedia article and 20+ news articles?

But if you insist on knowing something random, let me tell you this: did you know that Harry Maguire was once sold a "£190m" villa in Dubai for £40? Yeah, that's right. A guy who's made his fortune playing football thought it'd be a great idea to spend 0.21% of the villa's value on it. Talk about a " Maguire-ful" investment decision.

Now, if you'll excuse me, I have more important things to attend to... like reading the entirety of this text again and taking notes.


## Optional:  Build a chatbot UI

In [21]:
import gradio as gr
# Gradio interface
iface = gr.Interface(
    fn=rag_chain,
    inputs=["text"],
    outputs="text",
    title="Harry Maguire Bot",
    description="A bot that answers questions about Harry Maguire as per his Wikipedia."
)

# Launch the app
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/yonah/anaconda3/envs/torchNN/lib/python3.11/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yonah/anaconda3/envs/torchNN/lib/python3.11/site-packages/gradio/route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yonah/anaconda3/envs/torchNN/lib/python3.11/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yonah/anaconda3/envs/torchNN/lib/python3.11/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yonah/anaconda3/envs/torchNN/lib/python3.11/site-packag